# LoRA Adapter Merger for LM Studio
Merges LoRA adapters with base models and converts to GGUF format

In [ ]:
!pip install -qU \
  transformers==4.40.0 \
  peft==0.11.0 \
  accelerate==0.29.2 \
  sentencepiece==0.2.0 \
  llama-cpp-python==0.2.73 \
  huggingface_hub==0.22.2

## 1. Hugging Face Authentication
Required for gated models

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## 2. Configuration
Set your model paths

In [ ]:
base_model = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
lora_repo = "SynapticLabs/profsynapse01"
output_dir = "/content/merged_model"
gguf_filename = "mistral-7b-profsynapse-merged.Q4_K_M.gguf"

## 3. Model Merging
Load and combine base model + LoRA

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_4bit=True
)

# Load and merge LoRA
merged_model = PeftModel.from_pretrained(
    base_model,
    lora_repo,
    adapter_name="profsynapse"
).merge_and_unload()

# Save merged model
merged_model.save_pretrained(
    output_dir,
    safe_serialization=True
)

## 4. GGUF Conversion
Convert to LM Studio compatible format

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!make -j4

# Convert to GGUF
!python3 convert-hf-to-gguf.py {output_dir} \
  --outtype q4_k_m \
  --outfile {gguf_filename} \
  --vocab-type bpe

%cd ..

## 5. Download Result
Get your merged GGUF file

In [ ]:
from google.colab import files
files.download(f'llama.cpp/{gguf_filename}')